# I) Overview of ChIPseq Analysis
This pipeline is for SE ChIPseq data
This pipeline will do the following:

    1) Raw read QC (FastQC)
    2) Adapter trimming (cutadapt)
    3) Alignment (bowtie2)
    4) Mark duplicates (picard)
    5) Filtering to remove:
            #reads mapping to blacklisted regions (SAMtools, BEDTools)
            #reads that are marked as duplicates (SAMtools)
            #reads that arent marked as primary alignments (SAMtools)
            #reads that are unmapped (SAMtools)   
            #reads that map to multiple locations (SAMtools)
            #reads containing > 4 mismatches (SAMTools)
    6) MultiQC
    7) Create normalised bigWig files of the log2(IP/Input) (deepTools2)
    8) Call narrow peaks and generate bigWig files of the fold-enrichment of IP/Input (MACS2)
    9) Make a config file to use with for the differential peak calling with Diffbind.
    10) Generate pooled bigWig files of the log2(IP/Input) to use for heatmaps (deepTools2)

Input files:
   - fastq.gz files, one per each sample. This pipeline assumes that every IP has an Input
   - A config file. The config file is a tab seperated .txt file that contains 5 columns:
        - "seq_id": the initial sample id
        - "condition": sample type
        - "replicate": numeric
        - "mark": The histone modication profiled, or Input
        - "sample": new sample name
   - blacklist. Got hg19 from: https://github.com/Boyle-Lab/Blacklist/tree/master/lists

The working directory is "chipseq_data" and the genome path is /genome/hg19/hg19, change if using a different genome. Load both directories as volumes when launching Docker.

The hg19.chrom.sizes and blacklist region must be in the working directory


In [ ]:
import sys
import pandas as pd
from multiprocessing.pool import Pool
import os
import pybedtools
from pybedtools import BedTool
import itertools
import re

# II) Read in files and format config
- make group column (join mark + condition. eg CTRL_H3K4me3)
- make processed name: will be group + replicate + mark : sgNTC_1_H3K4me3

In [ ]:
config = pd.read_csv('config_file.txt', sep="\t")
config['group']=config['condition'] + "_" + config['mark']
config['processed_name']=config['condition'] + "_" + config['sample'] + "_" + config['mark']

## Specify all paths

In [ ]:
genome_path="/genome/hg19/hg19" # Change if need different genome/species
blacklist_filter= "/chipseq_data/blacklist_filter.bed"
fastq_path="/chipseq_data/fastq/"
working_dir="/chipseq_data/"

In [ ]:
'/Reference/KLF6/CRISPRa/ChIPseq/Deeptools/pooled/log2bw/'

## Make any directories that will be needed for output

In [ ]:
UMR_dir = 'UMR'
aligned_dir = 'Aligned_bam'
macs2_dir = 'macs2'
Deeptools_dir ='Deeptools'
Deeptools_log_dir='log2bw'
pooled_dir = 'pooled'
pooled_UMR_dir = 'UMR'


UMR_path = os.path.join(working_dir, UMR_dir)
aligned_path = os.path.join(working_dir, aligned_dir)
macs2_path = os.path.join(working_dir, macs2_dir)
Deeptools_path = os.path.join(working_dir, Deeptools_dir)
Deeptools_log_path = os.path.join(Deeptools_dir, Deeptools_log_dir)
pooled_path = os.path.join(working_dir, pooled_dir)
pooled_UMR_path = os.path.join(pooled_dir, pooled_UMR_dir)
pooled_Deep_path = os.path.join(pooled_dir, Deeptools_dir, Deeptools_log_dir)


os.makedirs(UMR_path)
os.makedirs(aligned_path)
os.makedirs(macs2_path)
os.makedirs(Deeptools_path)
os.makedirs(Deeptools_log_path)
os.makedirs(pooled_path)
os.makedirs(pooled_UMR_path)
os.makedirs(pooled_Deep_path)

# III) Main function

### Change to False in order to run!!

In [ ]:
DEBUG=True
#DEBUG=False

In [ ]:
def run_chipseq(x):
    
    read_length = 100 # change as appropriate
    
    # 1) Perform fastqc on fastq files
    
    fastq_fullpath=fastq_path + x['seq_id'] + '.fastq.gz'
    cmd1 = f'fastqc  --noexkst -f fastq -t 5 {fastq_fullpath}'

    print ('=' * 30)
    print (cmd1)
    if DEBUG == False:
        os.system(cmd1)
        
        
    # 2) Trim adapters and remove fragments <25 bp
    
    name=x['seq_id']
    cmd2 = f'cutadapt -l {read_length} -a AAGATCGGAAGAGCACACGTCTGAACTCCAGTCA -m 25 -o {fastq_path}{name}_trim.fastq.gz {fastq_fullpath} --cores=1'
    
    print ('=' * 30)
    print (cmd2)
    if DEBUG == False:
        os.system(cmd2)


    # 3) Align trimmed fastq to hg19 using bowtie
    #Save's alignment results to a log file
    trim_fullpath = fastq_path + name + '_trim.fastq.gz'
    aligned_name = aligned_path + "/" + x['seq_id'] + '.bam'
    cmd3 = f'(bowtie2 -p 5 -x {genome_path} -U {trim_fullpath}  | samtools view -Sb - > {aligned_name}) 2>{name}.log' 

    print ('=' * 30)
    print (cmd3)
    if DEBUG == False:
        os.system(cmd3)
        
        
    # 4) Coordinate sort .bam files
    
    aligned_sort_name = aligned_path + "/" + x['seq_id'] + '.sorted.bam'
    cmd4 = f'samtools sort {aligned_name} -@ 5 -o {aligned_sort_name}'
    
    print ('=' * 30)
    print (cmd4)
    if DEBUG == False:
        os.system(cmd4)
        
        
    # 5) Index sorted .bam files
    cmd5 = f'samtools index {aligned_sort_name}'

    print ('=' * 30)
    print (cmd5)
    if DEBUG == False:
        os.system(cmd5)
    
        
    # 6) Mark duplicates. 
    
    aligned_dup_name = aligned_path + "/" + x['seq_id'] + '.dedup.bam'
    marked_dup_metrics = x['seq_id'] + '_marked_dup_metrics.txt'
    cmd6 = f'picard MarkDuplicates I={aligned_sort_name} O={aligned_dup_name} M={marked_dup_metrics} ASSUME_SORTED=true  REMOVE_DUPLICATES=false VALIDATION_STRINGENCY=LENIENT'

    print ('=' * 30)                
    print (cmd6)
    if DEBUG == False:
        os.system(cmd6)
    
    
    # 7) Filter reads
    # Will filter out multimapped reads
    # Will filter out reads with quality less than 5 (-q 5). see http://dcc.blueprint-epigenome.eu/#/md/chip_seq_grch38
    # Will retain only mapped reads (-F 4)
    
    intermediate_bam = aligned_path + "/" + x['seq_id'] + '.intermediate.output.bam'
    cmd7 = f'samtools view -b -F 4 -q 5 {aligned_dup_name} > {intermediate_bam} && rm {aligned_dup_name}'
    
    print ('=' * 30)                
    print (cmd7)
    if DEBUG == False:
        os.system(cmd7)

    # 8) Remove duplicate reads  
    # Will filter out opitcal duplicates (Flag 1024)

    UMR_bam = UMR_path + "/" + x['seq_id'] + '.bam'
    cmd8 = f'samtools view -b -F 1024 -L {blacklist_filter} {intermediate_bam} > {UMR_bam} && rm {intermediate_bam}' 
    
    print ('=' * 30)                
    print (cmd8)
    if DEBUG == False:
        os.system(cmd8)
   
        
    # 9) Name sort the unique mapped reads
    UMR_sort_name= UMR_path + "/" + x['seq_id'] + '.sorted.bam'
    cmd9 = f'samtools sort {UMR_bam} -@ 5 -o {UMR_sort_name}'
    print ('=' * 30)
    print (cmd9)
    if DEBUG == False:
        os.system(cmd9)
    
    # 10) Index the sorted unique mapped reads

    cmd10 = f'samtools index {UMR_sort_name}'

    print ('=' * 30)
    print (cmd10)
    if DEBUG == False:
        os.system(cmd10)

#     print ('========end========')  

In [ ]:
PROCESSORS = 5 

In [ ]:
p_dup =Pool(PROCESSORS)

In [ ]:
p_dup.map(run_chipseq, [config.iloc[x] for x in range(config.shape[0])] ) 

## B) MultiQC

In [ ]:
!multiqc /chipseq_data/fastq/*_fastqc.zip --export

# IV) Generate log2 bigwigs
- These will be used for plotting heatmaps

In [ ]:
config['sample_condition'] = config['condition'] + "_" + config['sample'] 
config_ip = config.query('mark != "Input"').copy()
config_input = config.query('mark == "Input"').copy()
dict_samples_input = dict(zip(config_input['sample_condition'], config_input['seq_id']))
config_ip['input_id'] = config_ip['sample_condition'].apply(lambda x : dict_samples_input[x] if x in dict_samples_input else 'missing_wait_to_finish' )

In [ ]:
config_ip['bamReads'] = '/chipseq_data/UMR/' + config_ip['seq_id']  + '.sorted.bam'
config_ip['bamControl'] = '/chipseq_data/UMR/' + config_ip['input_id']  + '.sorted.bam'

In [ ]:
config_ip.head()

In [ ]:
for index, row in config_ip.iterrows():
    dir_loc= Deeptools_log_path + "/"
    config_ip['bamReads'] = UMR_dir + config_ip['seq_id']  + '.sorted.bam'
    config_ip['bamControl'] = UMR_dir + config_ip['input_id']  + '.sorted.bam'

    ip= row['bamReads']
    input_f = row['bamControl']
    name= row['processed_name']
    
    cmd1= f'cd {dir_loc}' 
    cmd2 = f'bamCompare --bamfile1 {ip} --bamfile2 {input_f} --outFileName {name}_log2.bw --operation log2  -p 15 --outFileFormat bigwig --scaleFactorsMethod readCount'
    cmd_to_run = cmd1 + ";" + cmd2
    
    #print(cmd_to_run)
    !$cmd_to_run

# V) macs2 peak calling

In [ ]:
def run_macs_callpeak(x):
    UMR_dir = UMR_path + "/"
    ip = UMR_dir + x['seq_id']  + '.sorted.bam'
    input_f = UMR_dir + x['input_id']  + '.sorted.bam'
    name = x['processed_name']
    macs_dir= macs2_path + "/"
    
    bdg_file=x['processed_name'] +'_FE.bdg'
    bdg_sort=x['processed_name'] +'_FE.sort.bdg'
    bw_file= macs_dir + x['processed_name'] +'_FE.bw'

    cmd_cd = f'cd {macs_dir}'
    cmd1 = f'macs2 callpeak -t {ip} -c {input_f} -f BAM -B -n {name} -g hs --nomodel -q 0.0001 --SPMR --keep-dup all --extsize 200'
    cmd2 = f'macs2 bdgcmp -t {name}_treat_pileup.bdg -c {name}_control_lambda.bdg -o {name}_FE.bdg -m FE'.format(name=x[0])
    cmd3 = f'cd {macs_dir}; sort -k1,1 -k2,2n {bdg_file} > {bdg_sort}; /UCSC/bedGraphToBigWig {bdg_sort} /chipseq_data/hg19.chrom.sizes {bw_file}'
    cmd_to_run = cmd_cd + ';' + cmd1 + ' ; ' + cmd2 + ' ; ' + cmd3

    print(cmd_to_run)
    
    if DEBUG == False:
        os.system(cmd_to_run)
        return(cmd_to_run)

In [ ]:
p_dup_macs =Pool(PROCESSORS)

In [ ]:
p_dup_macs.map(run_macs_callpeak, [config_ip.iloc[x] for x in range(config_ip.shape[0])] ) 

# VI) Make sample file for diffbind:

Sample sheet format according to diffbind:

data frame containing sample sheet, or file name of sample sheet to load (ignored if DBA is specified). Columns names in sample sheet may include:

    SampleID: Identifier string for sample

    Tissue: Identifier string for tissue type

    Factor: Identifier string for factor

    Condition: Identifier string for condition

    Treatment: Identifier string for treatment

    Replicate: Replicate number of sample

    bamReads: file path for bam file containing aligned reads for ChIP sample

    bamControl: file path for bam file containing aligned reads for control sample

    ControlID: Identifier string for control sample

    Peaks: path for file containing peaks for sample. format determined by PeakCaller field or caller parameter

    PeakCaller: Identifier string for peak caller used. If Peaks is not a bed file, this will determine how the Peaks file is parsed. If missing, will use default peak caller specified in caller parameter. Possible values:

        “raw”: text file file; peak score is in fourth column

        “bed”: .bed file; peak score is in fifth column

        “narrow”: default peak.format: narrowPeaks file

        “macs”: MACS .xls file

        “swembl”: SWEMBL .peaks file

        “bayes”: bayesPeak file

        “peakset”: peakset written out using pv.writepeakset

        “fp4”: FindPeaks v4

    PeakFormat: string indicating format for peak files; see PeakCaller and dba.peakset

    ScoreCol: column in peak files that contains peak scores

    LowerBetter: logical indicating that lower scores signify better peaks

    Counts: file path for externally computed read counts; see dba.peakset (counts parameter)

For sample sheets loaded from a file, the accepted formats are comma-separated values (column headers, followed by one line per sample), or Excel-formatted spreadsheets (.xls or .xlsx extension). Leading and trailing white space will be removed from all values, with a warning. 

In [ ]:
config_ip 

In [ ]:
def make_diffBind_sheet(x):
    UMR_dir = UMR_path + "/"
    macs_dir= macs2_path + "/"

    x['bamReads'] = UMR_dir + x['seq_id']  + '.sorted.bam'
    x['bamControl'] = UMR_dir + x['input_id']  + '.sorted.bam'
    x['peaks_path'] = macs_dir + x['processed_name'] + '_peaks.narrowPeak'
    
    sampleSheet = pd.DataFrame({'SampleID': x['processed_name'], 'Factor':x['mark'], 'Condition': x['condition'],
              'Treatment': x['condition'], 'Replicate': x['replicate'], 'bamReads':x['bamReads'],
             'bamControl' : x['bamControl'], 'ControlID':x['input_id'], 'Peaks' : x['peaks_path'], 'PeakCaller': 'macs'})
    print(sampleSheet)
    sampleSheet.to_csv('diffBind_sampleSheet.csv', sep = ',', index=False)

In [ ]:
make_diffBind_sheet(config_ip)

# VII) Bigwigs using pooled replicates
- Use these for heatmaps

## A) Generate pooled bams

In [ ]:
def bam_pool(x):
        pool_bam_dir = pooled_UMR_path + "/"
        UMR_dir = UMR_path + "/"

        reps_g = x.groupby('group')
        for k, v in reps_g:
            UMR_sort_name= UMR_dir + v['seq_id'] + '.sorted.bam'
            pool_sort_name= pool_bam_dir + k + '.sorted.bam'
            files_to_merge = ' '.join(UMR_sort_name.values.tolist())
            pool_name = pool_bam_dir + k + '.bam'
            cmd12 = f'samtools merge {pool_name} {files_to_merge} && samtools sort {pool_name} -@ 5 -o {pool_sort_name} && samtools index {pool_sort_name} && rm {pool_name}'

            !$cmd12
            print(cmd12)

In [ ]:
bam_pool(config)

## B) Make log2 bigwigs for the pooled bams

In [ ]:
config_ip['poolBamReads'] = pooled_UMR_path + "/" + config_ip['group']  + '.sorted.bam'
config_ip['poolBamControl'] = pooled_UMR_path + "/" + config_ip['condition']  + '_Input.sorted.bam'

In [ ]:
for index, row in config_ip.iterrows():
    dir_loc= pooled_Deep_path + "/"

    ip= row['poolBamReads']
    input_f = row['poolBamControl']
    name= row['poolBamReads']
    
    cmd1= f'cd {dir_loc}' 
    cmd2 = f'bamCompare --bamfile1 {ip} --bamfile2 {input_f} --outFileName {name}_log2.bw --operation log2  -p 15 --outFileFormat bigwig --scaleFactorsMethod readCount'
    cmd_to_run = cmd1 + ";" + cmd2
    
    print(cmd_to_run)
    !$cmd_to_run